In [ ]:
from dotenv import load_dotenv
import os
from hera.workflows import EmptyDirVolume, models, script, Artifact, Parameter, DAG, Steps, Step, NoneArchiveStrategy, Workflow
from hera.shared import global_config

load_dotenv("/home/otto/s1_zarr/.env")

In [ ]:
global_config.host = "https://dev.services.eodc.eu/workflows/"
global_config.namespace = "inca"
global_config.token = os.getenv("argo_token")
global_config.image = "ghcr.io/oscipal/image_zarr:latest"

In [ ]:
nfs_volume = [models.Volume(
    name="eodc-mount",
    persistent_volume_claim={"claimName": "eodc-nfs-claim"},
    )]

security_context = {"runAsUser": 59100,
                    "runAsGroup": 59100}

In [ ]:
@script(volume_mounts=[models.VolumeMount(name="eodc-mount", mount_path="/eodc")])
def write_to_volume():
    import pandas as pd

    data = {
    "A": [1, 2, 3, 4, 5],
    "B": [10, 20, 30, 40, 50],
    "C": [100, 200, 300, 400, 500]
    }

    df = pd.DataFrame(data)
    df.to_csv("/eodc/private/tempearth/data.csv")

In [ ]:
@script(volume_mounts=[models.VolumeMount(name="eodc-mount", mount_path="/eodc")])
def read_from_volume():
    import pandas as pd

    df = pd.read_csv("/eodc/private/tempearth/data.csv")
    print(df)

In [ ]:
with Workflow(
    generate_name="filesystem-access-",
    volumes = nfs_volume,
    security_context=security_context,
    entrypoint="workflow"
) as w:
    with DAG(name="workflow"):
        create = write_to_volume()
        read = read_from_volume()

        create >> read

In [ ]:
w.create()